In [2]:
!pip3 install -U pip
!pip3 install -U setuptools wheel
!pip3 install torch==1.13.1+cpu torchvision==0.14.1+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html
!pip3 install torch==1.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install autogluon
!pip install emoji
!pip install unidecode
!pip install Pillow==9.3.0
!pip install jedi==0.10
!pip install Pygments==2.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.5.1 which is incompatible.
L

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.8/186.8 kB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: Pygments
    Found existing installation: Pygments 2.14.0
    Uninstalling Pygments-2.14.0:
      Successfully uninstalled Pygments-2.14.0


In [1]:
#IMPORT
from google.colab import drive
import pandas as pd
import numpy as np

## 0. get files

In [2]:
drive.mount('/content/drive')

PATH = "/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/"

train = pd.read_csv(PATH+'data/train.csv')
test = pd.read_csv(PATH+'data/test.csv')
submission = pd.read_csv(PATH+'data/sample_submission.csv')

Mounted at /content/drive


## 1. Preprocessing

### 1-1. remove null/nan

In [3]:
print('1: ',len(train[train['AI']==1]))
print('0: ',len(train[train['AI']==0]))

1:  207
0:  539


In [4]:
print(train.isna().sum())
print(test.isna().sum())

ID        1
prompt    0
answer    1
AI        2
dtype: int64
ID        0
prompt    0
answer    0
dtype: int64


In [5]:
# col isna
for col in train.columns:
  display(train[train[col].isna()])

for col in train.columns:
  train = train[train[col].isna()==False]

,ID,prompt,answer,AI
610,NaN,0,NaN,NaN


,ID,prompt,answer,AI


,ID,prompt,answer,AI
610,NaN,0,NaN,NaN


,ID,prompt,answer,AI
609,785.0,define AI?,Artificial intelligence leverages computers an...,NaN
610,NaN,0,NaN,NaN


### 1-2. add wiki dataset

In [6]:
import json
import os

dataset_name = ['finance', 'open_qa', 'medicine', 'reddit_eli5', 'wiki_csai']
dataset_df = pd.DataFrame([])
for d_name in dataset_name:
  curl = 'curl -X GET "https://datasets-server.huggingface.co/first-rows?dataset=Hello-SimpleAI%2FHC3&config='+d_name+'&split=train"'
  
  # run curl comand and load response --> json dict --> df
  d_data = json.loads(os.popen(curl).read())

  cols = [x['name'] for x in d_data['features']]
  rows = [x['row'].values() for x in d_data['rows']]
  d_data = pd.DataFrame(rows, columns=cols)

  d_df = pd.DataFrame([])
  d_df['ID'] = [-1 for _ in range(len(d_data)*2)]
  d_df['prompt'] = list(d_data['question'])*2
  d_df['answer'] = [''.join(x) for x in d_data['human_answers']] + [''.join(x) for x in d_data['chatgpt_answers']]
  d_df['AI'] = [0 for _ in range(len(d_data))] + [1 for _ in range(len(d_data))]
  dataset_df = pd.concat([dataset_df, d_df])

### 1-3. Text Cleaning: remove symbols

In [8]:
import re
import emoji
from unidecode import unidecode

#remove html tags
def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)
    
#remove emoji and symbols
def dataPreprocessing(x): 
    x = x.lower()
    x = removeHTML(x)
    x = emoji.demojize(x, delimiters=(" ", " "))
    x = re.sub("'re", ' are',x) # 're n't --> are not 
    x = re.sub("n't", ' not',x) # 're n't --> are not 
    x = re.sub(r'\([^)]*\)', '' ,x)
    x = re.sub("/n",' ',x) 
    x = re.sub("/r",' ',x) 
    x = re.sub("\n",' ',x) 
    x = re.sub("\r",' ',x) 
    x = re.sub(r'[^A-Za-z0-9!.?, ]', '', x) 
    x = re.sub(r'[,]',' ',x)
    x = re.sub("[ ]+",' ',x) # space*n --> space
    x = re.sub("_",'',x) # _people_ --> people
    x = x.strip()
    return unidecode(x) #emily bronté --> emily bronte, mi² --> mi2, πr --> pr

In [9]:
train_text = train.copy()
train_text = pd.concat([train_text, dataset_df])
train_text["prompt"]=train_text["prompt"].apply(lambda x: dataPreprocessing(x))
train_text["answer"]=train_text["answer"].apply(lambda x: dataPreprocessing(x))

In [10]:
test_text = test.copy()
test_text["prompt"]=test["prompt"].apply(lambda x: dataPreprocessing(x))
test_text["answer"]=test["answer"].apply(lambda x: dataPreprocessing(x))

In [11]:
print(len(train_text))
print(len(test_text))

1474
249


In [12]:
#check
for col in ['prompt', 'answer']:
  print([x for x in train_text[col] if len(list(filter(re.compile(r'[^A-Za-z0-9 ]').match,x)))>0])

['what is the future of ai?', 'what is your biggest challenge in your career?', 'what is the tallest mountain in the world?', 'what are the best 5 super cars?', 'what is deep learning?', 'what is the currency of the european union?', 'what is your opinion on the current state of renewable energy?', 'what is your favorite movie?', 'how do you succeed in digital marketing?', 'what is the most meaningful relationship in your life?', 'how do i become a millionaire?', 'what is the most meaningful thing in your life?', 'do you believe in ghosts?', 'how do i prevent scrum from turning great developers into average developers?', 'what is the best way to improve your selfesteem?', 'what is the best structure for a team ?', 'what are the most important skills for a technical career?', 'what is the importance of ai in healthcare?', 'is social media marketing more important than email marketing?', 'can you define friendly ai?', 'can you tell me the name of the tallest waterfall in the world?', 'wh

In [14]:
train_text

,ID,prompt,answer,AI
0,690.0,what is the future of ai?,the future of artificial intelligence is both ...,1.0
1,304.0,what is your biggest challenge in your career?,my biggest challenge in my career is balancing...,0.0
2,63.0,what is the tallest mountain in the world?,the tallest mountain in the world is mount eve...,1.0
3,894.0,what are the best 5 super cars?,mclaren 720s. ferrari 296 gtb. audi r8. masera...,0.0
4,796.0,what is deep learning?,a type of machine learning based on artificial...,0.0
...,...,...,...,...
149,-1.0,please explain what is optimization theory,optimization theory is a branch of mathematics...,1.0
150,-1.0,please explain what is naturallanguage user in...,a naturallanguage user interface is a type of ...,1.0
151,-1.0,please explain what is information retrieval,information retrieval is the process of search...,1.0
152,-1.0,please explain what is question answering,question answering is a type of natural langua...,1.0


### 1-4. train_test_split

In [54]:
from sklearn.model_selection import train_test_split

X = train_text.drop(['ID'], axis=1)
y = train_text['AI']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train), len(X_test))

1179 295


In [55]:
X_train

,prompt,answer,AI
94,what is extreme right wing,extreme right wing refers to a political ideol...,1.0
254,what is robotics?,robotics is a branch of engineering that invol...,0.0
58,how to increase my serum albumin levels?i have...,dear i read your scenario .ppi do nothing to y...,0.0
645,what is the name of the largest mammal in the ...,the blue whale,1.0
133,please explain what is fuzzy system,a fuzzy system is a computational system that ...,1.0
...,...,...,...
114,q. what is the treatment for lv diastolic dysf...,lv diastolic dysfunction refers to a condition...,1.0
94,why are stormclouds gray when normal clouds ar...,storm clouds are gray because they contain a l...,1.0
114,would i ever need credit card if my debit card...,it is generally not necessary to have a credit...,1.0
139,please explain what is web ontology language,web ontology language is a language for repres...,1.0


In [56]:
train_data = pd.concat([pd.DataFrame(X_train, columns=['prompt', 'answer']), pd.DataFrame(y_train, columns=['AI'])], axis=1)
train_data

,prompt,answer,AI
94,what is extreme right wing,extreme right wing refers to a political ideol...,1.0
254,what is robotics?,robotics is a branch of engineering that invol...,0.0
58,how to increase my serum albumin levels?i have...,dear i read your scenario .ppi do nothing to y...,0.0
645,what is the name of the largest mammal in the ...,the blue whale,1.0
133,please explain what is fuzzy system,a fuzzy system is a computational system that ...,1.0
...,...,...,...
114,q. what is the treatment for lv diastolic dysf...,lv diastolic dysfunction refers to a condition...,1.0
94,why are stormclouds gray when normal clouds ar...,storm clouds are gray because they contain a l...,1.0
114,would i ever need credit card if my debit card...,it is generally not necessary to have a credit...,1.0
139,please explain what is web ontology language,web ontology language is a language for repres...,1.0


In [57]:
test_data = pd.concat([pd.DataFrame(X_test, columns=['prompt', 'answer']), pd.DataFrame(y_test, columns=['AI'])], axis=1)
test_data

,prompt,answer,AI
106,can expense ratios on investment options in a ...,yes expense ratios on investment options in a ...,1.0
184,whats the purpose of openai?,openais mission is to ensure that artificial g...,0.0
61,if salt is so bad for cars why do we use it on...,salt is used on roads to help melt ice and sno...,1.0
67,what is the use of react hooks in react?,react hooks are a way to manage state and comp...,0.0
220,what is the most important quality for a leader?,the most important quality for a leader in my ...,0.0
...,...,...,...
560,what is computer vision?,computer vision is used to detect and classify...,0.0
42,what is the role of heredity,heredity is the passing of traits to offspring...,0.0
522,what is the meaning of life?,the meaning of life is subjective and can vary...,0.0
128,where to park money while saving for a car,there are a few options you might consider whe...,1.0


## 2. Modeling

In [58]:
label = 'AI'
eval_metric = 'acc'
time_limit = 3600 * 2

In [59]:
from datetime import datetime

now = datetime.now()
EPOCH = 50
MODELNAME = 'autoGluon_QA_EPOCH'+str(EPOCH)+'_CHA_'+str(now.year)+str(now.month)+str(now.day)+str(now.hour)+str(now.minute)
MODELNAME

'autoGluon_QA_EPOCH50_CHA_2023371338'

In [60]:
from ray import tune

In [61]:
hyperparameters = {
        # "optimization.learning_rate": tune.uniform(0.00005, 0.005),
        # "optimization.optim_type": tune.choice(["adamw", "sgd"]),
        # "optimization.max_epochs": tune.choice(["40", "50"]),
        # "model.timm_image.checkpoint_name": tune.choice(["swin_base_patch4_window7_224", "convnext_base_in22ft1k"])
}
hyperparameter_tune_kwargs = {
    # "searcher": "bayes", # random
    # "scheduler": "ASHA",
    # "num_trials": 2,
}

In [62]:
import os
import ray


In [63]:
from autogluon.multimodal import MultiModalPredictor
import uuid

predictor = MultiModalPredictor(label=label, eval_metric=eval_metric)
predictor.fit(train_data, presets='best_quality', 
              hyperparameters={"optimization.max_epochs": EPOCH, "optimization.patience": 100},
              time_limit=time_limit)

INFO:lightning_fabric.utilities.seed:Global seed set to 123
No path specified. Models will be saved in: "AutogluonModels/ag-20230307_133846/"
AutoMM starts to create your model. ✨

- Model will be saved to "/content/AutogluonModels/ag-20230307_133846".

- Validation metric is "acc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20230307_133846
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens 

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 3: 'val_acc' reached 0.39407 (best 0.39407), saving model to '/content/AutogluonModels/ag-20230307_133846/epoch=0-step=3.ckpt' as top 3


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 39, global step 319: 'val_acc' was not in top 3
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Start to fuse 3 checkpoints via the greedy soup algorithm.


Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

Predicting: 0it [00:00, ?it/s]

AutoMM has created your model 🎉🎉🎉

- To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/content/AutogluonModels/ag-20230307_133846")
    ```

- You can open a terminal and launch Tensorboard to visualize the training log:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /content/AutogluonModels/ag-20230307_133846
    ```

- If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub: https://github.com/autogluon/autogluon




## 3. Test

In [73]:
test_score = predictor.evaluate(test_data)
print(test_score)

Predicting: 0it [00:00, ?it/s]

{'acc': 0.9355932203389831}


In [74]:
# model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_text)

Predicting: 0it [00:00, ?it/s]

In [75]:
submission['Category'] = model_pred

In [76]:
submission = submission.astype({'Category':'int64'})

In [77]:
submission

,Id,Category
0,710,1
1,487,0
2,136,1
3,44,1
4,627,1
...,...,...
244,702,0
245,500,0
246,818,0
247,584,0


In [78]:
submission.to_csv(PATH+'submit/autoGluon_QA_'+MODELNAME+'.csv', index=False)

# 4. Save Best Model

In [79]:
model = predictor.dump_model()

import pickle

with open(PATH+'model/'+MODELNAME+'.pkl', 'wb') as f:
    pickle.dump(model, f)

from joblib import dump, load

dump(model, PATH+'model/'+MODELNAME+'.joblib')

Model weights for hf_text are saved to /content/AutogluonModels/ag-20230307_133846/hf_text.
Tokenizer hf_text saved to /content/AutogluonModels/ag-20230307_133846/hf_text.


['/content/drive/MyDrive/ML-Olympiad/Detect-ChatGpt-answers/model/autoGluon_QA_EPOCH50_CHA_2023371338.joblib']